In [1]:
SELECT *
FROM InductionCompleted

(0 rows affected)

Total execution time: 00:00:00.090

InductionCompletedID,PersonProjectId,CompletedDate,Title,Email,ProjectName


InductionCompleted table store information about users who have completed all compulsory trainings. 

It is updated by triger inductionCompleted\_00, on CompletedTrainings Tab

<span style="color: var(--vscode-foreground);">Up to the project requirements Table has to be updated with name, email, project name<br></span>

In [4]:
ALTER TABLE InductionCompleted
ADD Title NVARCHAR(100),
    Email NVARCHAR(100), 
    ProjectName NVARCHAR(100);

Commands completed successfully.

Total execution time: 00:00:00.064

In [19]:
ALTER TRIGGER inductionCompleted_00
ON CompletedTrainings
FOR INSERT AS
BEGIN 
	
	DECLARE @PersonProjectID int
	DECLARE @ProjectID int
    DECLARE @PersonID int
	DECLARE @NumCompletedTrainings int
	DECLARE @NumTrainingsProject int
    DECLARE @Title nvarchar(100)
    DECLARE @Email nvarchar(100) 
    DECLARE @ProjectName nvarchar(100)

	SET @PersonProjectID = (SELECT PersonProjectID FROM inserted)

	SET @ProjectID = (SELECT ProjectID FROM PersonProject WHERE PersonProjectID = @PersonProjectID)
    SET @PersonID = (SELECT PersonID FROM PersonProject WHERE PersonProjectID = @PersonProjectID)

	SET @Title = (SELECT Title FROM People_Details WHERE PersonDetailsID = @PersonID)
    SET @Email = (SELECT Email FROM People_Details WHERE PersonDetailsID = @PersonID) 
    SET @ProjectName = (SELECT ProjectName FROM Projects WHERE ProjectID = @ProjectID)     

	SET @NumTrainingsProject = (SELECT COUNT(*) FROM Trainings WHERE ProjectID = @ProjectID AND Mandatory = 1)
	SET @NumCompletedTrainings = (SELECT COUNT(*) FROM CompletedTrainings ct JOIN Trainings t ON ct.TrainingID = t.TrainingID WHERE t.Mandatory = 1 AND ct.PersonProjectID = @PersonProjectID)

	IF @NumCompletedTrainings >= @NumTrainingsProject
		BEGIN

		INSERT INTO InductionCompleted (PersonProjectId, CompletedDate, Title, Email, ProjectName)
		VALUES (@PersonProjectID, CAST(GETDATE() AS DATE), @Title, @Email, @ProjectName)

		END

END
GO

Commands completed successfully.

Total execution time: 00:00:00.048

<span style="color: rgb(0, 128, 0);">defining groups with people details view for flow</span>

In [5]:
ALTER VIEW PeopleInGroups
AS      
    SELECT pd.Title, pd.Email, p_g.GroupUrl, p_g.PortalGroupName, p.ProjectName
    FROM PersonGroup pg
    JOIN Portal_Groups p_g
    ON pg.GroupID = p_g.PortalGroupID
    JOIN PersonProject pp
    ON pg.PersonProjectID = pp.PersonProjectID
    JOIN People_Details pd 
    ON pp.PersonID = pd.PersonDetailsID
    JOIN Projects p 
    ON pp.ProjectID = p.ProjectID

Commands completed successfully.

Total execution time: 00:00:00.065

In [6]:
SELECT *
FROM PeopleInGroups

(6 rows affected)

Total execution time: 00:00:00.061

Title,Email,GroupUrl,PortalGroupName,ProjectName
Leigh Hulme,leigh.hulme@jacobs.com,NULL,Dev Test,EssexCCFW
Grzogorz Brzęszczyszczykiweicz,grzegorz.brzeczyszczczykiewicz@samsiegubie.com,NULL,Dev Test,EssexCCFW
Grzogorz Brzęszczyszczykiweicz,grzegorz.brzeczyszczczykiewicz@samsiegubie.com,NULL,Dev Test 2,EssexCCFW
Grzogorz Brzęszczyszczykiweicz,grzegorz.brzeczyszczczykiewicz@samsiegubie.com,NULL,Dev Test 2,EssexCCFW
Harry Potter,harry.potter@hogward.com,NULL,Dev Test 2,EssexCCFW
Paulina Haber,paulina.haber@jacobs.com,NULL,Dev Test 2,EssexCCFW
